<a href="https://colab.research.google.com/github/LC1332/Embed-Adapter/blob/main/notebook/Embed_Adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我打算先建立一个 openAI(预先存储), 和bge-medium和bge-small之间相互转化的代码

- [x] 从google drive进行数据读取
- [x] 进行batch循环，调通基础的batch embedding抽取
- [x] 判断哪些模型之间的数据是需要被记录的，记录x'x和x'y
- [ ] 计算伪逆并进行初步的存储


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/unpublic

ChatHaruhi_Waifu_extended.jsonl  erotics.jsonl	zhwiki_2k_embedding.jsonl  星绘.jsonl


这段代码是特别处理之前我们在训练LuotuoBert的时候使用的left-right格式的代码

下载链接

https://drive.google.com/file/d/1JLJ68Xs67ZihZnygohwDtJ2uKEIEuuu1/view?usp=sharing

In [34]:
fname = "/content/drive/MyDrive/unpublic/zhwiki_2k_embedding.jsonl"

import json

raw_datas = []

with open(fname, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip() == "":
            continue
        data = json.loads(line)
        raw_datas.append(data)

raw_datas = raw_datas[:100]

In [ ]:
print(raw_datas[0].keys())
print(raw_datas[0])

In [36]:
datas = []
for data in raw_datas:
    datas.append({
        "text": data["left"],
        "embedding": data["left_embedding"],
    })
    datas.append({
        "text": data["right"],
        "embedding": data["right_embedding"],
    })

In [37]:
print(len(datas))

200


把bge家族的代码放过来

TODO: 把这个代码移动到repo里

In [38]:
from tqdm import tqdm
import torch

_model_pool = {}
_tokenizer_pool = {}

# BAAI/bge-small-zh-v1.5

def get_general_embeddings( sentences , model_name = "BAAI/bge-small-zh-v1.5" , return_tensor = False):

    global _model_pool
    global _tokenizer_pool

    if model_name not in _model_pool:
        from transformers import AutoTokenizer, AutoModel
        _tokenizer_pool[model_name] = AutoTokenizer.from_pretrained(model_name)
        _model_pool[model_name] = AutoModel.from_pretrained(model_name)

    _model_pool[model_name].eval()

    # Tokenize sentences
    encoded_input = _tokenizer_pool[model_name](sentences, padding=True, truncation=True, return_tensors='pt', max_length = 512)

    # Compute token embeddings
    with torch.no_grad():
        model_output = _model_pool[model_name](**encoded_input)
        # Perform pooling. In this case, cls pooling.
        sentence_embeddings = model_output[0][:, 0]

    # normalize embeddings
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    if return_tensor == True:
        return sentence_embeddings
    return sentence_embeddings.cpu().tolist()

def get_general_embedding( text_or_texts , model_name = "BAAI/bge-small-zh-v1.5" ):
    if isinstance(text_or_texts, str):
        return get_general_embeddings([text_or_texts], model_name)[0]
    else:
        return get_general_embeddings_safe(text_or_texts, model_name)

general_batch_size = 16

import math

def get_general_embeddings_safe(sentences, model_name = "BAAI/bge-small-zh-v1.5"):

    embeddings = []

    num_batches = math.ceil(len(sentences) / general_batch_size)

    for i in tqdm( range(num_batches) ):
        # print("run bge with batch ", i)
        start_index = i * general_batch_size
        end_index = min(len(sentences), start_index + general_batch_size)
        batch = sentences[start_index:end_index]
        embs = get_general_embeddings(batch, model_name)
        embeddings.extend(embs)

    return embeddings

def get_bge_zh_embedding( text_or_texts ):
    return get_general_embedding(text_or_texts, "BAAI/bge-small-zh-v1.5")

对embedding函数的测试

测试这两个

BAAI/bge-small-zh-v1.5

BAAI/bge-base-zh-v1.5

In [10]:
# test_batch_n = 4

# test_batch = [data["text"] for data in datas[:test_batch_n]]

# bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)

# bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

X_1 = bge_small_15 是一个 4 * 512的tensor

X_2 = bge_base_15 是一个 4 * 768的tensor

我想 1. 验证X_i 是否每行都被l_2归一化

2. 计算X_i^T X_i，以及X_1^T X_2的值，

请用python为我实现

我们现在开始来正规写程序

In [40]:
texts = [data["text"] for data in datas]
openai_embeddings = [data["embedding"] for data in datas]

In [41]:
openai_config = {
    "name":"openai",
    "embeddings":openai_embeddings, # 预先抽取的
    "batch_embed_fun":None
}

def bge_small_zh_fun( texts ):
    return get_general_embeddings(texts, "BAAI/bge-small-zh-v1.5", return_tensor = True)

bge_small_config = {
    "name":"bge_small_zh_15",
    "embeddings":None,
    "batch_embed_fun":bge_small_zh_fun
}

def bge_base_zh_fun( texts ):
    return get_general_embeddings(texts, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_base_config = {
    "name":"bge_base_zh_15",
    "embeddings":None,
    "batch_embed_fun":bge_base_zh_fun
}

method_configs = [openai_config, bge_small_config, bge_base_config]

forbidden_pairs = [] # 如果你某个pair不希望生成，需要把 model_A_2_model_B 放到这个list里面

In [42]:
n = len(texts)

n_method = len(method_configs)
batch_size = 8

from tqdm import tqdm

corr_map = {}

for start_id in tqdm(range(0,n, batch_size)):
    end_id = min(start_id + batch_size, n)
    texts_batch = texts[start_id:end_id]

    method2embeddings = {}


    for method_config in method_configs:
        method_name = method_config["name"]
        if method_config["embeddings"] is None:
            embeddings = method_config["batch_embed_fun"](texts_batch)
        else:
            embeddings = torch.tensor(method_config["embeddings"][start_id:end_id]).cpu()

        if embeddings.device != torch.device("cpu"):
            embeddings = embeddings.cpu()

        method2embeddings[method_name] = embeddings

    for method_i in range(n_method):
        for method_j in range(method_i, n_method):
            corr_index = ( method_i, method_j )
            X_1 = method2embeddings[method_configs[method_i]["name"]]
            X_2 = method2embeddings[method_configs[method_j]["name"]]
            X1TX2 = X_1.T @ X_2
            if corr_index in corr_map:
                corr_map[corr_index] += X1TX2
            else:
                corr_map[corr_index] = X1TX2

for method_i in range(n_method - 1):
    for method_j in range(method_i + 1, n_method):
        corr_index = ( method_i, method_j )
        trans_index = (method_j, method_i )
        corr_map[trans_index] = corr_map[corr_index].T

100%|██████████| 25/25 [03:58<00:00,  9.53s/it]


In [44]:
import numpy as np

pseudo_inverses = {}

for i in range(n_method):
    # 计算每个方法自身相关性矩阵的伪逆
    corr_ii = corr_map[(i, i)].numpy()
    pseudo_inv_ii = np.linalg.pinv(corr_ii)

    for j in range(n_method):
        if i != j:
            # 计算交叉相关性矩阵
            corr_ij = corr_map[(i, j)].numpy()
            # 计算伪逆
            pseudo_inverse_ij = pseudo_inv_ii @ corr_ij
            pseudo_inverses[(i, j)] = pseudo_inverse_ij

# 现在 pseudo_inverses 包含了所有相关性矩阵的伪逆


TODO:

- [ ] 写一下伪逆的存储
- [ ] 适当用更多的数据（600条）进行训练后，验证
- [ ] 验证能不能放到hf

In [30]:
a = {(1,2):3, (2,4):5}

In [ ]:
print(a)

In [29]:
print(method2embeddings["openai"].shape)

torch.Size([8, 1536])
